# Data Quality Assessment Notebook

## 1. Setup and Imports

In [ ]:
import commons
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

# Pandas Variables
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 40)

# Save location for plots
USERS_NULL_PCT_CHART_PATH = os.path.join(
    commons.PLOTS_PATH, "null_percentages", "users.png"
)
TRANSACTIONS_NULL_PCT_CHART_PATH = os.path.join(
    commons.PLOTS_PATH, "null_percentages", "transactions.png"
)
PRODUCTS_NULL_PCT_CHART_PATH = os.path.join(
    commons.PLOTS_PATH, "null_percentages", "products.png"
)

In [ ]:
# Reading Data files
products = pd.read_parquet(commons.PRODUCTS_CLEAN_PARQUET_PATH)
users = pd.read_parquet(commons.USERS_CLEAN_PARQUET_PATH)
transactions = pd.read_parquet(commons.TRANSACTIONS_CLEAN_PARQUET_PATH)

## 2. Data Overview 
This notebook analyzes data quality across three key datasets:
- Products table
- Users table  
- Transactions table

### 2.1 Missing Data Analysis
The below code examines missing values and their distributions across all three tables. The results here are discussed in Section 3.2 in the Quality Analysis doc.

In [ ]:
# Helpful functions
def get_null_counts(df: pd.DataFrame):
    return pd.DataFrame(
        {
            "column": df.columns,
            "null_count": df.isnull().sum(),
            "non_null_count": df.notnull().sum(),
        }
    ).reset_index(drop=True)


def plot_null_percentages(df: pd.DataFrame):
    # Calculate percentages
    df_sorted = df.sort_values("null_count", ascending=False)
    total = df_sorted["null_count"] + df_sorted["non_null_count"]
    df_sorted["null_pct"] = (df_sorted["null_count"] / total * 100).round(1)
    df_sorted["non_null_pct"] = (df_sorted["non_null_count"] / total * 100).round(1)

    plt.figure(figsize=(12, 6))

    sns.barplot(x="column", y="null_pct", data=df_sorted, color="red", label="Null %")
    sns.barplot(
        x="column",
        y="non_null_pct",
        data=df_sorted,
        color="green",
        label="Non-Null %",
        bottom=df_sorted["null_pct"],
    )

    # Add percentage labels
    for i, pct in enumerate(df_sorted["null_pct"]):
        plt.text(i, 50, f"{pct}%", ha="center", va="center")

    plt.xticks(rotation=45, ha="right")
    plt.xlabel("Column Names")
    plt.ylabel("Percentage (%)")
    plt.ylim(0, 100)
    plt.legend()
    plt.tight_layout()
    return plt


# === Code ===
products_nulls = get_null_counts(products)
users_nulls = get_null_counts(users)
transactions_nulls = get_null_counts(transactions)

plot = plot_null_percentages(transactions_nulls)
plot.title("Transactions Table")
plot.savefig(TRANSACTIONS_NULL_PCT_CHART_PATH, dpi=300, format="png")

plot = plot_null_percentages(users_nulls)
plot.title("Users Table")
plot.savefig(USERS_NULL_PCT_CHART_PATH, dpi=300, format="png")

plot = plot_null_percentages(products_nulls)
plot.title("Products Table")  # MIs
plot.savefig(PRODUCTS_NULL_PCT_CHART_PATH, dpi=300, format="png")

### 2.3 Data Integrity Analysis

#### 2.3.1 Duplicate Entries 
Identifies completely duplicate rows in each table.

In [ ]:
# Helpful functions
def get_num_redundant_rows(df: pd.DataFrame, pk_col: str):
    duplicate_rows = df[df.duplicated(keep=False)]
    # Calculate metrics
    unique_dupl_rows = len(duplicate_rows[pk_col].unique())
    total_duplicate_rows = len(duplicate_rows)
    return total_duplicate_rows - unique_dupl_rows


# === Code ===
print(
    "Redundant rows in {0}: {1}".format(
        "transactions", get_num_redundant_rows(transactions, "RECEIPT_ID")
    )
)
print(
    "Redundant rows in {0}: {1}".format(
        "products", get_num_redundant_rows(products, "BARCODE")
    )
)
print("Redundant rows in {0}: {1}".format("users", get_num_redundant_rows(users, "ID")))

#### 2.3.2 Inconsistent Entries
This analysis identifies records where the same key value (e.g., BARCODE, USER_ID, RECEIPT_ID) appears multiple times but with different values in other columns. These are different from pure duplicates because some data fields don't match.


In [ ]:
# Helpful functions
def get_inconsistent_entries(df: pd.DataFrame, pk_col: str):
    clean_df = df[df[pk_col].notna()].copy()
    duplicate_mask = clean_df.duplicated(subset=[pk_col], keep=False)
    inconsistent_entries = clean_df[duplicate_mask].sort_values(pk_col)
    return inconsistent_entries.sort_values(by=pk_col)


# === Code ===
products_inconsistent = get_inconsistent_entries(products, "BARCODE")
users_inconsistent = get_inconsistent_entries(users, "ID")
transactions_inconsistent = get_inconsistent_entries(transactions, "RECEIPT_ID")

print(
    "Number of inconsistent records in {0}: {1}".format(
        "products", len(products_inconsistent)
    )
)
print(
    "Number of inconsistent records in {0}: {1}".format(
        "transactions", len(transactions_inconsistent)
    )
)
print(
    "Number of inconsistent records in {0}: {1}".format(
        "users", len(users_inconsistent)
    )
)

### 2.4 Data Reference Analysis

#### 2.4.1 Barcode Reference Check
This analysis examines referential integrity between transactions and products tables by validating barcode relationships.

**Purpose:**
- Verify that every barcode in transactions table has a corresponding entry in products table
- Identify orphaned transactions (those with barcodes not found in products)
- Calculate the percentage of transactions with invalid barcode references

In [ ]:
# Remove nulls from `BARCODE`
cleaned_transactions = transactions[transactions["BARCODE"].notna()].copy()

# Find barcodes in transactions that don't exist in products catalog
missing_barcodes = cleaned_transactions[
    ~cleaned_transactions["BARCODE"].isin(products["BARCODE"])
]

# Get unique counts for calculation
unique_barcodes = cleaned_transactions["BARCODE"].unique()
unique_missing_barcodes = missing_barcodes["BARCODE"].unique()

# Print summary
print(f"Total missing barcodes: {len(unique_barcodes)}")
print(
    f"Percentage missing: {(len(unique_missing_barcodes)/len(unique_barcodes))*100:.2f}%"
)

#### 2.4.2 User Reference Check

Analyzes referential integrity between transactions and users tables by verifying user ID relationships.

**Purpose:**
- Verify that every `USER_ID` in `transactions` table has a corresponding entry in `users` table
- Identify orphaned users (those with barcodes not found in `users`)
- Calculate the percentage of transactions with invalid barcode references

In [ ]:
# Find IDs in transactions that don't exist in users table 
missing_users = transactions[~transactions["USER_ID"].isin(users["ID"])][
   "USER_ID"
].unique()

# Get count of all unique users in transactions
unique_users = transactions["USER_ID"].unique()

# Print summary stats
print(f"Total missing users: {len(missing_users)}")
print(f"Percentage missing: {(len(missing_users)/len(unique_users))*100:.2f}%")

### 3.3.4 Data Validation Anomalies 

Checks for records that violate basic business logic - in this case, transactions where receipt was scanned before purchase date.

In [ ]:
# 3.3.4. Validation Issues
print(
    "Number of records in transactions where `SCAN_DATE` < 'PURCHASE_DATE`: {}".format(
        len(transactions[transactions["SCAN_DATE"] < transactions["PURCHASE_DATE"]])
    )
)